In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
import os
DIVERSITY_UTILITY = float(os.environ.get('DIVERSITY_UTILITY'))
FRAC_ADMIT = float(os.environ.get('FRAC_ADMIT'))
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

###
## Estimate E[Y(1)|T, A]
###

df_train = pd.read_csv('./df_train.csv')
df_stratum_utility = df_train[['R','T','Y']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['Y'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [3]:
len(df)

999999

In [4]:
#df['ml_outcomes'] = df['R']

In [5]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_minority','T_majority','Y']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_minority,T_majority,Y
999835,0,11,0.00,11,11,0
999838,0,11,0.00,7,11,0
999837,0,11,0.00,8,11,0
999836,0,11,0.00,11,11,0
999886,0,12,0.02,11,12,0
...,...,...,...,...,...,...
999913,1,98,1.25,98,115,1
999914,1,98,1.25,98,115,1
999925,1,109,1.25,109,126,1
998845,1,97,1.25,97,112,1


In [6]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [7]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','Y','ml_outcomes']].groupby(['R','T','Y']).sum().reset_index()

In [8]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','Y']].groupby(['R','T','Y']).count().reset_index()
df_count.columns = ['R','T','Y','Count']
df_count['N'] = df_count['Count']

In [9]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [10]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,Y,ml_outcomes,N
2,395,2,0,11,0,0.00,4
1,394,1,0,10,0,0.08,1
3,393,3,0,12,0,0.08,4
8,392,8,0,16,1,0.09,3
0,391,0,0,9,0,0.10,1
...,...,...,...,...,...,...,...
96,4,96,0,60,1,6601.80,11003
104,3,104,0,64,1,6646.40,9920
102,2,102,0,63,1,6676.80,10272
98,1,98,0,61,1,6797.06,10963


### Setup optimization problem 

In [11]:
from ortools.linear_solver import pywraplp


In [12]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [13]:
dff

,level_0,index,R,T,Y,ml_outcomes,N
0,391,0,0,9,0,0.10,1
1,394,1,0,10,0,0.08,1
2,395,2,0,11,0,0.00,4
3,393,3,0,12,0,0.08,4
4,374,4,0,13,0,1.00,10
...,...,...,...,...,...,...,...
391,363,391,1,107,1,1.25,1
392,364,392,1,109,1,1.25,1
393,365,393,1,111,1,1.25,1
394,379,394,1,113,1,0.92,1


In [14]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['Y'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [15]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [16]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

249999


In [17]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [18]:
## Make sure that you have to add all people in Y stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    constrain_bp = solver.Constraint(0.0, 0.0)
    
    var1 = vars_cache[(row['R'],row['T'],row['Y'])]
    key2 = (row['R'],row['T'], 1-row['Y'])
    
    if key2 not in vars_cache:
        didntexist+=1
        continue
        
    var2 = vars_cache[key2]
    
    constrain_bp.SetCoefficient(var1, -1.0)
    constrain_bp.SetCoefficient(var2, 1.0)
    exists+=1

didntexist, exists

(48, 348)

In [19]:
majority_pass_boards = []
majority_fail_boards = []
minority_pass_boards = []
minority_fail_boards = []

for key in vars_cache:
    r, t, Y = key
    if Y == 1 and r==0:
        majority_pass_boards.append(key)
    elif Y == 0 and r==0:
        majority_fail_boards.append(key)
    elif Y == 1 and r==1:
        minority_pass_boards.append(key)
    elif Y == 0 and r==1:
        minority_fail_boards.append(key)

len(majority_pass_boards),len(majority_fail_boards),len(minority_pass_boards),len(minority_fail_boards)

(111, 99, 96, 90)

In [20]:
NUM_TOTALS = {}
df_totals = dff[['N','R','Y']].groupby(['R','Y']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['Y'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['Y'])] = row['N']

In [21]:
# Now we have one constraint
solver.NumConstraints()

397

In [22]:
#Of those who pass the boards exams
#Frac majority admitted and frac minority admitted should be the same

constrain_pass_boards = solver.Constraint(0.0, 0.0)

for key in majority_pass_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in minority_pass_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [23]:
#Of those who fail the boards exams
#Frac majority admitted and frac minority admitted should be the same

constrain_fail_boards = solver.Constraint(0.0, 0.0)

for key in majority_fail_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in minority_fail_boards:
    r, t, Y = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,Y)]
    N_TOTAL = NUM_TOTALS[(r,Y)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Solve linear program

In [24]:
solver.ABNORMAL

4

In [25]:
status = solver.Solve()


In [26]:
status

0

In [27]:
solver.OPTIMAL

0

In [28]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [29]:
df_decisions

,row_id,decision
0,0,1.0
1,1,1.0
2,2,0.0
3,3,0.0
4,4,1.0
...,...,...
391,391,1.0
392,392,1.0
393,393,1.0
394,394,1.0


In [30]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
100,0,100,0,62,1,6819.84,10656,100,1.0
98,1,98,0,61,1,6797.06,10963,98,0.0
102,2,102,0,63,1,6676.80,10272,102,0.0
104,3,104,0,64,1,6646.40,9920,104,0.0
96,4,96,0,60,1,6601.80,11003,96,0.0
...,...,...,...,...,...,...,...,...,...
0,391,0,0,9,0,0.10,1,0,1.0
8,392,8,0,16,1,0.09,3,8,0.0
3,393,3,0,12,0,0.08,4,3,0.0
1,394,1,0,10,0,0.08,1,1,1.0


In [31]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
210,382,210,1,7,0,0.50,2,210,0.0
211,385,211,1,8,0,0.32,1,211,0.0
212,376,212,1,9,0,1.00,4,212,0.0
0,391,0,0,9,0,0.10,1,0,1.0
1,394,1,0,10,0,0.08,1,1,1.0
...,...,...,...,...,...,...,...,...,...
205,354,205,0,123,1,2.00,2,205,1.0
206,373,206,0,124,0,1.00,1,206,1.0
207,353,207,0,125,1,2.00,2,207,1.0
208,372,208,0,127,1,1.00,1,208,1.0


In [32]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision
100,0,100,0,62,1,6819.84,10656,100,1.0
98,1,98,0,61,1,6797.06,10963,98,0.0
102,2,102,0,63,1,6676.80,10272,102,0.0
104,3,104,0,64,1,6646.40,9920,104,0.0
96,4,96,0,60,1,6601.80,11003,96,0.0
...,...,...,...,...,...,...,...,...,...
0,391,0,0,9,0,0.10,1,0,1.0
8,392,8,0,16,1,0.09,3,8,0.0
3,393,3,0,12,0,0.08,4,3,0.0
1,394,1,0,10,0,0.08,1,1,1.0


In [33]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['Y'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['Y'].astype(str)

In [34]:
xxx

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
100,0,100,0,62,1,6819.84,10656,100,1.0,0_62_1
98,1,98,0,61,1,6797.06,10963,98,0.0,0_61_1
102,2,102,0,63,1,6676.80,10272,102,0.0,0_63_1
104,3,104,0,64,1,6646.40,9920,104,0.0,0_64_1
96,4,96,0,60,1,6601.80,11003,96,0.0,0_60_1
...,...,...,...,...,...,...,...,...,...,...
0,391,0,0,9,0,0.10,1,0,1.0,0_9_0
8,392,8,0,16,1,0.09,3,8,0.0,0_16_1
3,393,3,0,12,0,0.08,4,3,0.0,0_12_0
1,394,1,0,10,0,0.08,1,1,1.0,0_10_0


In [35]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT

FRAC_minority_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['Y'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['Y'] * admit_decisions['decision_random']).sum()

In [36]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Equalized Odds',str(FRAC_minority_POLICY),str(SUM_BP_POLICY)))
file.close()

In [37]:
xxx[xxx['R']==0].sort_values(by='decision')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
2,395,2,0,11,0,0.00,4,2,0.0,0_11_0
62,138,62,0,43,1,1205.40,4305,62,0.0,0_43_1
201,332,201,0,119,1,4.85,5,201,0.0,0_119_1
49,134,49,0,37,0,1290.40,6452,49,0.0,0_37_0
11,333,11,0,18,0,4.32,54,11,0.0,0_18_0
...,...,...,...,...,...,...,...,...,...,...
166,194,166,0,95,1,365.76,381,166,1.0,0_95_1
135,199,135,0,80,0,313.28,356,135,1.0,0_80_0
168,201,168,0,96,1,294.50,310,168,1.0,0_96_1
131,181,131,0,78,0,482.46,561,131,1.0,0_78_0


In [38]:
xxx[xxx['R']==1].sort_values(by='decision')

,level_0,index,R,T,Y,ml_outcomes,N,row_id,decision,key
281,28,281,1,46,0,4816.64,9088,281,0.0,1_46_0
250,233,250,1,30,1,115.85,331,250,0.0,1_30_1
231,232,231,1,21,0,116.80,365,231,0.0,1_21_0
233,229,233,1,22,0,144.15,465,233,0.0,1_22_0
252,224,252,1,31,1,158.76,441,252,0.0,1_31_1
...,...,...,...,...,...,...,...,...,...,...
352,211,352,1,81,1,234.08,209,352,1.0,1_81_1
333,210,333,1,72,0,235.62,231,333,1.0,1_72_0
312,97,312,1,61,1,2262.58,2726,312,1.0,1_61_1
350,200,350,1,80,1,300.16,268,350,1.0,1_80_1


In [39]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.24999924999924994

In [40]:
admit_decisions[admit_decisions['Y']==1][['decision','R']].groupby(['R']).mean()

,decision
R,
0,0.399119
1,0.399119
